## AirBnD Data load

NOTE:

In this notebook im only selecting, casting, exploding cols (with fixed structure and size), adding additional geo data and imputing missing vals.

I moved all of this to a proper airbnb_data_loader.py file where we can load the data and use it further for creating embeddings themselves.
We can further improve stuff there - like imputation (maybe there is a better technique or optimized method with regressors I didn't know about) or new cols we want to use (i tried to make it modular overall)



---


I've continued to work in airbnb_embeddings_eda.ipynb where i load the data (same logic as in this ipynb), transform it and try to do some initial attempts at embeddings

In [0]:
storage_account = "lab94290"  
container = "airbnb"

In [0]:
sas_token="sp=rle&st=2025-12-24T17:37:04Z&se=2026-02-28T01:52:04Z&spr=https&sv=2024-11-04&sr=c&sig=a0lx%2BS6PuS%2FvJ9Tbt4NKdCJHLE9d1Y1D6vpE1WKFQtk%3D"
sas_token = sas_token.lstrip('?')
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)

In [0]:
path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/airbnb_1_12_parquet"

airbnb = spark.read.parquet(path)
display(airbnb.limit(5))

In [0]:
files = dbutils.fs.ls("dbfs:/Users")
for f in files:
    print(f.name)

In [0]:
path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
print(f"{path}")

In [0]:
def get_folder_size(path):
    """Recursively get total size of all files in folder"""
    total_size = 0
    try:
        files = dbutils.fs.ls(path)
        for f in files:
            if f.isDir():
                # Recursively get subdirectory size
                total_size += get_folder_size(f.path)
            else:
                total_size += f.size
    except Exception as e:
        print(f"Error reading {path}: {e}")
    return total_size

total_bytes = get_folder_size(path)
size_gb = total_bytes / (1024**3)
print(f"Total size: {size_gb:.2f} GB ({total_bytes:,} bytes)")

In [0]:
airbnb.printSchema()

In [0]:
airbnb.count()


In [0]:
airbnb.count(), airbnb.select("property_id").distinct().count()


#### Selecting necessary columns and casting them

In [0]:
from pyspark.sql.functions import col, regexp_replace

airbnb_sel = airbnb.select(
    "property_id",
    "listing_name",
    "listing_title",

    "lat",
    "long",

    "ratings",
    "reviews",
    "property_number_of_reviews",

    "host_rating",
    "host_number_of_reviews",
    "host_response_rate",
    "hosts_year",
    "is_supperhost",
    "is_guest_favorite",

    "guests",

    "category",
    "category_rating",

    "amenities",
    "description",
    "description_items",
    "details",
    "arrangement_details",

    "pricing_details",
    "total_price",
    "currency",

    "availability",
    "final_url"
)

display(airbnb_sel.limit(10))

In [0]:
from pyspark.sql.functions import col, split, trim, lower, size

# Removed location logic since we enrich data with reverse geocoder
airbnb_cast = airbnb_sel\
    .withColumn("lat", col("lat").cast("double"))\
    .withColumn("long", col("long").cast("double"))\
    .filter(col("lat").isNotNull() & col("long").isNotNull())\
    .withColumn("ratings", col("ratings").cast("double"))\
    .withColumn("property_number_of_reviews", col("property_number_of_reviews").cast("int"))\
    .withColumn("host_rating", col("host_rating").cast("double"))\
    .withColumn("host_number_of_reviews", col("host_number_of_reviews").cast("int"))\
    .withColumn("host_response_rate", col("host_response_rate").cast("double"))\
    .withColumn("hosts_year", col("hosts_year").cast("int"))\
    .withColumn("total_price", col("total_price").cast("double"))\
    .withColumn("guests", col("guests").cast("int"))\
    .withColumn("is_supperhost", (col("is_supperhost") == "true").cast("int"))\
    .withColumn("is_guest_favorite", (col("is_guest_favorite") == "true").cast("int"))\
    .withColumn("is_available", (col("availability") == "true").cast("int"))\
    .dropDuplicates(["property_id"])

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType
from pyspark.sql.functions import from_json, col, map_from_entries, element_at

# Define the specific Schemas for the JSON blobs
# We define these manually to ensure correct data types (e.g., handling nulls and doubles)

# Schema for pricing_details (simple struct)
pricing_schema = StructType([
    StructField("airbnb_service_fee", DoubleType()),
    StructField("cleaning_fee", DoubleType()),
    StructField("initial_price_per_night", DoubleType()),
    StructField("num_of_nights", IntegerType()),
    StructField("price_per_night", DoubleType()),
    StructField("price_without_fees", DoubleType()),
    StructField("special_offer", DoubleType()),
    StructField("taxes", DoubleType())
])

# Schema for category_rating (array of structs)
# The raw data has "value" as a string ("5.0"), so we parse as string first
category_schema = ArrayType(StructType([
    StructField("name", StringType()),
    StructField("value", StringType())
]))

# Apply parsing and extracting
airbnb_features = airbnb_cast \
    .withColumn("pricing_struct", from_json(col("pricing_details"), pricing_schema)) \
    .withColumn("category_struct", from_json(col("category_rating"), category_schema)) \
    .select(
        "*", 
        # Flatten Pricing: This promotes pricing_struct.cleaning_fee to a top-level column named 'cleaning_fee'
        "pricing_struct.*" 
    ) \
    .drop("pricing_struct", "pricing_details", "category_rating") # Clean up raw cols

# Handle Category Ratings
# The category_rating is a list like [{"name": "Cleanliness", "value": "5.0"}, ...].
# We need to turn this into a Map so we can pick out "Cleanliness" easily.

# Convert Array<Struct> to Map<String, Double>
airbnb_features = airbnb_features.withColumn(
    "ratings_map", 
    map_from_entries(col("category_struct"))
)

# Extract specific ratings into their own columns and cast to Double
# We use element_at to grab the value by its key name
airbnb_features_ext = airbnb_features \
    .withColumn("rating_cleanliness", element_at(col("ratings_map"), "Cleanliness").cast("double")) \
    .withColumn("rating_accuracy", element_at(col("ratings_map"), "Accuracy").cast("double")) \
    .withColumn("rating_checkin", element_at(col("ratings_map"), "Check-in").cast("double")) \
    .withColumn("rating_communication", element_at(col("ratings_map"), "Communication").cast("double")) \
    .withColumn("rating_location", element_at(col("ratings_map"), "Location").cast("double")) \
    .withColumn("rating_value", element_at(col("ratings_map"), "Value").cast("double")) \
    .drop("category_struct", "ratings_map")

#### Adding geo data with revese geocoder

In [0]:
!pip install reverse_geocoder

In [0]:
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
import reverse_geocoder as rg

# Initialize
print("Initializing reverse geocoder...")
rg.search((0, 0))
print("Ready!")

# Schema for geocoding results - matching POI format
geo_schema = StructType([
    StructField("name", StringType()),        # city name
    StructField("cc", StringType()),          # country code
    StructField("admin1", StringType()),      # state/province
    StructField("admin2", StringType()),      # county/region
])

@pandas_udf(geo_schema)
def reverse_geocode_udf(lat_series: pd.Series, long_series: pd.Series) -> pd.DataFrame:
    """Batch reverse geocode lat/long coordinates"""
    # Convert strings to floats, handle nulls
    lats = pd.to_numeric(lat_series, errors='coerce')
    longs = pd.to_numeric(long_series, errors='coerce')
    
    # Create coords list, skip invalid ones
    coords = []
    valid_indices = []
    for i, (lat, lon) in enumerate(zip(lats, longs)):
        if pd.notna(lat) and pd.notna(lon):
            coords.append((lat, lon))
            valid_indices.append(i)
    
    # Geocode valid coords
    if coords:
        results = rg.search(coords, mode=2)
    else:
        results = []
    
    # Build output dataframe
    output = []
    result_idx = 0
    for i in range(len(lat_series)):
        if i in valid_indices:
            r = results[result_idx]
            output.append({
                "name": r.get("name"),
                "cc": r.get("cc"),
                "admin1": r.get("admin1"),
                "admin2": r.get("admin2"),
            })
            result_idx += 1
        else:
            # Invalid coords - return nulls
            output.append({
                "name": None,
                "cc": None,
                "admin1": None,
                "admin2": None,
            })
    
    return pd.DataFrame(output)

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
import pandas as pd

def analyze_missing_values(df: DataFrame, show_all: bool = False) -> pd.DataFrame:
    """
    Analyze missing values in a PySpark DataFrame.
    
    Parameters:
    -----------
    df : DataFrame
        PySpark DataFrame to analyze
    show_all : bool, default=False
        If True, show all columns. If False, only show columns with missing values.
    
    Returns:
    --------
    pd.DataFrame
        Summary of missing values with styling
    """
    print("Analyzing missing values...")
    
    # Calculate missing counts
    missing_counts = df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c)
        for c in df.columns
    ]).toPandas()
    
    # Create summary dataframe
    total_count = df.count()
    missing_df = pd.DataFrame({
        'column': missing_counts.columns,
        'missing': missing_counts.iloc[0].values,
    })
    missing_df['missing_pct'] = (missing_df['missing'] / total_count * 100).round(2)
    missing_df['present'] = total_count - missing_df['missing']
    missing_df['present_pct'] = (missing_df['present'] / total_count * 100).round(2)
    
    # Sort by missing percentage
    missing_df = missing_df.sort_values('missing_pct', ascending=False)
    
    # Filter if needed
    if not show_all:
        missing_df = missing_df[missing_df['missing'] > 0]
    
    missing_df = missing_df.reset_index(drop=True)
    
    # Print summary
    cols_with_missing = (missing_df['missing'] > 0).sum()
    print(f"\nTotal rows: {total_count:,}")
    print(f"Total columns: {len(df.columns)}")
    print(f"Columns with missing values: {cols_with_missing}\n")
    
    # Color function
    def color_missing(val):
        if val < 1:
            return 'background-color: lightgreen'
        elif val < 5:
            return 'background-color: yellow'
        elif val < 30:
            return 'background-color: orange'
        else:
            return 'background-color: red'
    
    # Style and return
    styled_df = missing_df.style.applymap(color_missing, subset=['missing_pct'])
    
    return styled_df

In [0]:
# Apply geocoding and extract fields with addr_ prefix (matching POI format)
airbnb_geo = airbnb_features_ext.withColumn(
    "geo_data",
    reverse_geocode_udf(col("lat"), col("long"))
).select(
    "*",  # Keep all original columns
    col("geo_data.name").alias("addr_name"),      # City name
    col("geo_data.cc").alias("addr_cc"),          # Country code
    col("geo_data.admin1").alias("addr_admin1"),  # State/Province
    col("geo_data.admin2").alias("addr_admin2")   # County/Region
).drop("geo_data")  # Drop the temporary struct column


In [0]:

display(analyze_missing_values(airbnb_geo))

#### Dropping where perct of missing cols is extremely small

In [0]:
from pyspark.sql.functions import col

print("Dropping where perct of missing cols is extremely small")

airbnb_filtered = airbnb_geo.filter(
    col("property_id").isNotNull() &
    col("listing_name").isNotNull() &
    col("listing_title").isNotNull() &
    col("ratings").isNotNull() &
    col("is_guest_favorite").isNotNull() &
    col("guests").isNotNull() &
    col("amenities").isNotNull() &
    col("pricing_details").isNotNull() &
    col("details").isNotNull() &
    col("description_items").isNotNull()
)
print(f"Dropped: {airbnb_features_ext.count() - airbnb_filtered.count():,} rows")

#### Filling rows with appropriate vals

In [0]:
from pyspark.sql.functions import col, when, coalesce, lit

print("Filling missing values in numeric/categorical columns...")

airbnb_filled = airbnb_filtered.withColumn(
    "property_number_of_reviews",
    coalesce(col("property_number_of_reviews"), lit(0))
).withColumn(
    "host_number_of_reviews",
    coalesce(col("host_number_of_reviews"), lit(0))
).withColumn(
    "is_supperhost",
    coalesce(col("is_supperhost"), lit(0))  # Not superhost if missing
).withColumn(
    "is_available",
    coalesce(col("is_available"), lit(0))  # Not available if missing
).withColumn(
    "cleaning_fee", 
    coalesce(col("cleaning_fee"), lit(0.0)) # No fees if missing
).withColumn(
    "airbnb_service_fee", 
    coalesce(col("airbnb_service_fee"), lit(0.0)) # No fees if missing
).withColumn(
    "num_of_nights", 
    coalesce(col("num_of_nights"), lit(1)) # base nightly rate
).withColumn(
    "taxes", 
    coalesce(col("taxes"), lit(0.0)) # No tax if missing
).withColumn(
    "special_offer", 
    coalesce(col("special_offer"), lit(0.0)) # No special offer if missing
)

#### Deriving prices

In [0]:
df_math = airbnb_filled.withColumn(
    "derived_price_per_night",
    (
        col("total_price") 
        - col("airbnb_service_fee") 
        - col("cleaning_fee") 
        - col("taxes") 
        - col("special_offer") # If this is -37, subtracting it adds 37 back to base
    ) / col("num_of_nights")
)

df_reconstructed = df_math.withColumn(
    "final_price_per_night",
    coalesce(
        col("price_per_night"),           # Trust explicit price
        col("initial_price_per_night"),   # Trust explicit initial price
        col("price_without_fees") / col("num_of_nights"), # Trust explicit base price
        col("derived_price_per_night"),   # Use our reverse-engineered math
        col("total_price") / col("num_of_nights") # Last resort: just divide total by nights
    )
).withColumn(
    "final_num_of_nights",
    col("num_of_nights") # We use our safe version (defaults to 1)
)

# Round to 2 decimals and ensure no negatives (e.g., if fees > total due to data error)
df_reconstructed = df_reconstructed.withColumn(
    "final_price_per_night", 
    when(col("final_price_per_night") < 0, col("total_price")) # Fallback
    .otherwise(round(col("final_price_per_night"), 2))
).withColumn(
    "final_num_of_nights", 
    col("final_num_of_nights")
)

#### Filling text cols appropriately

In [0]:
from pyspark.sql.functions import col, coalesce, lit, when, array

airbnb_txt = df_reconstructed.withColumn(
    "description",
    coalesce(
        col("description"), 
        col("listing_title"), 
        col("listing_name"), 
        lit("No description provided")
    )
)

#### Converting eur to usd

In [0]:
from pyspark.sql.functions import col, when

# Convert EUR to USD, keep USD as-is, then drop old columns
airbnb_usd = airbnb_txt.withColumn(
    "derived_price_per_night",
    when(col("currency") == "USD", col("derived_price_per_night"))
    .when(col("currency") == "EUR", col("derived_price_per_night") * 1.08) # Using this num to convert EUR to USD
    .otherwise(None)
)

In [0]:
display(analyze_missing_values(airbnb_usd))

#### Imputing price, host/category ratings...

In [0]:
from pyspark.sql.functions import avg, col, coalesce, lit, broadcast, round

# Columns to impute based on Country context
country_cols = [
    "host_rating", "host_response_rate", "hosts_year",
    "rating_cleanliness", "rating_accuracy", "rating_checkin", 
    "rating_communication", "rating_location", "rating_value", 
    "price_per_night"
]

# Create expression list for efficient aggregation: [avg(col1), avg(col2)...]
agg_exprs = [round(avg(c), 2).alias(f"avg_{c}") for c in country_cols]


# CALCULATE STATISTICS 
print("Calculating Country & Global Statistics...")

# Country Stats (Group by Country)
# One pass to get averages for all columns for every country
country_stats = airbnb_usd.groupBy("addr_cc").agg(*agg_exprs)

# Global Stats (Scalar Fallbacks)
# One pass to get the global average for all columns as fallback
global_stats_row = airbnb_usd.agg(*agg_exprs).collect()[0]

global_defaults = {}
for c in country_cols:
    val = global_stats_row[f"avg_{c}"]
    global_defaults[c] = val

In [0]:
display(country_stats)

In [0]:
global_defaults

In [0]:
# Join the country stats once (Broadcasted for speed)
df_impute_ready = airbnb_usd.join(
    broadcast(country_stats), 
    on="addr_cc", 
    how="left"
)

df_imputed = df_impute_ready

# Loop through columns and apply the Waterfall Logic
for c in country_cols:
    df_imputed = df_imputed.withColumn(
        c,
        coalesce(
            col(c),                  # 1. Original Value
            col(f"avg_{c}"),         # 2. Country Average
            lit(global_defaults[c])  # 3. Global Average Fallback
        )
    ).drop(f"avg_{c}") # Drop temp column immediately to keep schema clean

In [0]:
from pyspark.sql.functions import col, regexp_replace

airbnb_final = df_imputed.select(
    "property_id",
    "listing_name",
    "listing_title",

    "lat", 
    "long", 
    "addr_name",            # Enriched City (e.g., "Broadbeach")
    "addr_admin1",          # Enriched State (e.g., "Queensland")
    "addr_cc",              # Enriched Country (e.g., "AU")

    "ratings",
    # "reviews", # no reviews???
    "property_number_of_reviews",

    "host_rating",
    "host_number_of_reviews",
    "host_response_rate",
    "hosts_year",
    "is_supperhost",
    "is_guest_favorite",

    "category",
    "rating_cleanliness",
    "rating_accuracy",
    "rating_checkin",
    "rating_communication",
    "rating_location",
    "rating_value",

    "amenities",
    "description",
    "description_items",
    "details",
    # "arrangement_details", # there is details, i think its unnecessary

    "price_per_night",      # The base unit for comparison (Imputed)
    "num_of_nights",        # The stay duration base (Imputed)
    "guests",               # Capacity (Cast to int)

    "is_available",
    "final_url"
)

In [0]:
display(airbnb_final.limit(30))

In [0]:
display(analyze_missing_values(airbnb_final))

In [0]:
# from pyspark.sql.functions import col, coalesce, lit

# print("Filling missing text columns...")

# airbnb_filled = airbnb_filled.withColumn(
#     "reviews",
#     # Check for both NULL and empty list string "[]"
#     when(
#         (col("reviews").isNull()) | (col("reviews") == "[]"), 
#         lit('["No reviews"]')
#     ).otherwise(col("reviews"))
# ).withColumn(
#     "description",
#     coalesce(col("description"), lit("No description provided"))
# ).withColumn(
#     "arrangement_details",
#     # Check for both NULL and empty list string "[]"
#     when(
#         (col("arrangement_details").isNull()) | (col("arrangement_details") == "[]"), 
#         lit('[{"name": "Arrangement", "value": "Not specified"}]')
#     ).otherwise(col("arrangement_details"))
# )

# print("Text columns filled")

In [0]:
# from pyspark.sql.functions import col

# # List of columns that contain JSON data
# json_cols = [
#     "pricing_details",      # Expecting {}
#     "category_rating",      # Expecting []
#     "reviews",              # Expecting []
#     "amenities",            # Expecting []
#     "arrangement_details",  # Expecting []
#     "description_items"     # Expecting []
#     "details"               # Expecting []
# ]

# print("--- Data Quality Check: Missing or Empty JSON ---")

# for c in json_cols:
#     # Check if column exists in DataFrame
#     if c in airbnb_filled.columns:
#         # Count 1: Actual Nulls
#         null_count = airbnb_filled.filter(col(c).isNull()).count()
        
#         # Count 2: Empty Lists "[]"
#         empty_list_count = airbnb_filled.filter(col(c) == "[]").count()
        
#         # Count 3: Empty Objects "{}"
#         empty_obj_count = airbnb_filled.filter(col(c) == "{}").count()
        
#         if null_count > 0 or empty_list_count > 0 or empty_obj_count > 0:
#             print(f"Column: {c}")
#             print(f"  - NULLs: {null_count}")
#             print(f"  - Empty Lists '[]': {empty_list_count}")
#             print(f"  - Empty Objects '{{}}': {empty_obj_count}")
#             print("-" * 20)